<a href="https://colab.research.google.com/github/aswini03Aswini/Unhack/blob/main/Milestone1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install opencv-python-headless pandas

In [6]:
import zipfile
import os

# Unzip the dataset
zip_path = '/content/DataSet.zip'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/dataset')

# List the files
dataset_path = '/content/dataset'
print(os.listdir(dataset_path))


['DataSet']


In [27]:
import cv2
import pandas as pd
import os

def load_templates(template_folder):
    templates = {}
    for filename in os.listdir(template_folder):
        if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".PNG"):
            template_path = os.path.join(template_folder, filename)
            templates[filename] = cv2.imread(template_path, 0)
    return templates

def match_template(image, templates):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    detected_speed = None
    max_val = 0

    for template_name, template in templates.items():
        result = cv2.matchTemplate(gray_image, template,cv2.TM_CCOEFF_NORMED)
        _, max_val_curr, _, _ = cv2.minMaxLoc(result)

        if max_val_curr > max_val:
            max_val = max_val_curr
            detected_speed = template_name

    if detected_speed:
        return detected_speed.split('.')[0]  # Assuming the template filenames are the speed limits (e.g., '30.jpg')
    return None

def process_images(input_folder, template_folder, output_csv):
    templates = load_templates(template_folder)
    results = []

    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(input_folder, filename)
            image = cv2.imread(image_path)
            #image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
            speed_limit = match_template(image, templates)
            results.append([filename, speed_limit[9:]])

    df = pd.DataFrame(results, columns=["Input image name", "Speed limit"])
    df.to_csv(output_csv, index=False)

# Define your paths

input_folder='/content/dataset/DataSet/Milestone 2'
template_folder='/content/dataset/DataSet/Template images'
output_csv='M1_Output.csv'

# Run the process
process_images(input_folder, template_folder, output_csv)



In [26]:
import cv2
import pandas as pd
import os
import numpy as np

def load_templates(template_folder):
    templates = {}
    for filename in os.listdir(template_folder):
        if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".PNG"):
            template_path = os.path.join(template_folder, filename)
            template = cv2.imread(template_path, 0)
            if template is not None:
                templates[filename] = template
            else:
                print(f"Warning: Failed to load template {filename}")
    return templates

def preprocess_image(image):
    if image is None:
        raise ValueError("Invalid image input for preprocessing")
    #gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 200)
    return edges

def match_template(image, templates):
    if image is None:
        raise ValueError("Invalid image input for matching")
    image_edges = preprocess_image(image)
    detected_speed = None
    max_val = 0

    for template_name, template in templates.items():
        template_edges = preprocess_image(template)
        for angle in range(-10, 11, 2):  # Rotate templates from -10 to 10 degrees
            M = cv2.getRotationMatrix2D((template.shape[1] // 2, template.shape[0] // 2), angle, 1)
            rotated_template = cv2.warpAffine(template_edges, M, (template.shape[1], template.shape[0]))
            result = cv2.matchTemplate(image_edges, rotated_template, cv2.TM_CCOEFF_NORMED)
            _, max_val_curr, _, _ = cv2.minMaxLoc(result)

            if max_val_curr > max_val:
                max_val = max_val_curr
                detected_speed = template_name

    if detected_speed:
        return detected_speed.split('.')[0]  # Assuming the template filenames are the speed limits (e.g., '30.jpg')
    return None

def process_images(input_folder, template_folder, output_csv):
    templates = load_templates(template_folder)
    results = []

    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(input_folder, filename)
            image = cv2.imread(image_path)
            if image is None:
                print(f"Warning: Failed to load image {filename}")
                continue
            speed_limit = match_template(image, templates)
            if speed_limit is not None:
                results.append([filename, speed_limit])
            else:
                print(f"Warning: No speed limit detected for image {filename}")

    df = pd.DataFrame(results, columns=["Input image name", "Speed limit"])
    df.to_csv(output_csv, index=False)

# Define your paths
input_folder = '/content/dataset/DataSet/Milestone 2'
template_folder = '/content/dataset/DataSet/Template images'
output_csv = 'M1_Output.csv'

# Run the process
process_images(input_folder, template_folder, output_csv)


In [35]:
import cv2
import pandas as pd
import os
import re

def load_templates(template_folder):
    templates = {}
    for filename in os.listdir(template_folder):
        if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".PNG"):
            template_path = os.path.join(template_folder, filename)
            template = cv2.imread(template_path, 0)
            if template is not None:
                templates[filename] = template
            else:
                print(f"Warning: Failed to load template {filename}")
    return templates

def extract_speed_from_filename(filename):
    match = re.search(r'\d+', filename)
    if match:
        return int(match.group(0))
    return None

def preprocess_image(image):
    if image is None or image.size == 0:
        raise ValueError("Invalid image input for preprocessing")
    if len(image.shape) == 2:
        gray = image
    elif len(image.shape) == 3 and image.shape[2] == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        raise ValueError("Unsupported image format")

    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 200)
    return edges

def detect_vehicles(image):
    # Placeholder function to detect vehicles based on bounding boxes
    # This assumes bounding boxes are annotated in some way within the image.
    # You would replace this with actual code to read annotations or detect vehicles.
    # For now, we use dummy values:
    return [(100, 100, 150, 150), (200, 200, 250, 250)]  # Dummy bounding boxes

def match_template(image, templates):
    if image is None or image.size == 0:
        raise ValueError("Invalid image input for matching")
    image_edges = preprocess_image(image)
    detected_speed = None
    max_val = 0

    for template_name, template in templates.items():
        template_edges = preprocess_image(template)
        for angle in range(-10, 11, 2):
            M = cv2.getRotationMatrix2D((template.shape[1] // 2, template.shape[0] // 2), angle, 1)
            rotated_template = cv2.warpAffine(template_edges, M, (template.shape[1], template.shape[0]))
            if rotated_template.shape[0] > image_edges.shape[0] or rotated_template.shape[1] > image_edges.shape[1]:
                continue
            result = cv2.matchTemplate(image_edges, rotated_template, cv2.TM_CCOEFF_NORMED)
            _, max_val_curr, _, _ = cv2.minMaxLoc(result)

            if max_val_curr > max_val:
                max_val = max_val_curr
                detected_speed = template_name

    if detected_speed:
        speed = extract_speed_from_filename(detected_speed)
        if speed is not None:
            return speed
    return None

def process_images(input_folder, template_folder, output_csv):
    templates = load_templates(template_folder)
    results = []

    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(input_folder, filename)
            image = cv2.imread(image_path)
            if image is None or image.size == 0:
                print(f"Warning: Failed to load image {filename}")
                continue
            try:
                speed_limit = match_template(image, templates)
                if speed_limit is not None:
                    vehicle_boxes = detect_vehicles(image)
                    vehicle_count = len(vehicle_boxes)
                    overspeeding_count = sum(1 for box in vehicle_boxes if get_vehicle_speed(image, box) > speed_limit)
                    results.append([filename, speed_limit, vehicle_count, overspeeding_count])
                else:
                    print(f"Warning: No speed limit detected for image {filename}")
            except ValueError as e:
                print(f"Error processing image {filename}: {e}")

    df = pd.DataFrame(results, columns=["Input image name", "Speed limit", "No. of vehicles", "No. of over speeding vehicles"])
    df.to_csv(output_csv, index=False)

def get_vehicle_speed(image, bbox):
    # Dummy function to return vehicle speed. Replace with actual logic to read speed from annotations.
    return 70  # Dummy speed

# Define your paths
input_folder = '/content/dataset/DataSet/Milestone 3'
template_folder = '/content/dataset/DataSet/Template images'
output_csv = 'M3_Output.csv'

# Run the process
process_images(input_folder, template_folder, output_csv)


In [37]:
!pip install pytesseract

In [39]:
!apt-get update
!apt-get install -y tesseract-ocr


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [861 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get